In [2]:
!pip install torch torchvision tqdm
import torch, os
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

    100% |████████████████████████████████| 519.5MB 32kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x58860000 @  0x7f7e468d11c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 1.7MB/s 
    100% |████████████████████████████████| 51kB 9.6MB/s 
    100% |████████████████████████████████| 2.0MB 2.4MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [3]:
transform = transforms.Compose([transforms.ToTensor(),])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified


In [0]:
def train(epoch, trainloader, optimizer, criterion):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader), 0):
        # get the inputs
        inputs, labels = data
        if torch.cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

    print('epoch %d training loss: %.3f' %
            (epoch + 1, running_loss / (len(trainloader))))

In [0]:
def test(testloader, model):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
                                    100 * correct / total))

In [0]:
def classwise_test(testloader, model):
########################################################################
# class-wise accuracy

    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data
            if torch.cuda.is_available():
                images, labels = images.cuda(), labels.cuda()        
            outputs = net(images)
            _, predicted = torch.max(outputs, 1)
            c = (predicted == labels).squeeze()
            for i in range(4):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    for i in range(10):
        print('Accuracy of %5s : %2d %%' % (
            classes[i], 100 * class_correct[i] / class_total[i]))


In [9]:
num_epochs = 5
learning_rate = 0.001
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=256, kernel_size=5)#32-5+1=28
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#14
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=1024, kernel_size=3)#14-3+1=12
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)#6
        self.conv3 = nn.Conv2d(in_channels=1024, out_channels=2048, kernel_size=3)#6-3+1=4
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#2
        self.conv4 = nn.Conv2d(in_channels=2048, out_channels=2048, kernel_size=1)#2
        self.pool4= nn.MaxPool2d(kernel_size=2, stride=2)#1
        self.fc1 = nn.Linear(in_features=2048*1*1, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=10)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = self.pool4(F.relu(self.conv4(x)))
        x = x.view(-1, 2048*1*1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
net = Net()
# transfer the model to GPU
if torch.cuda.is_available():
    net = net.cuda()
    print("Using GPU")

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)#, weight_decay=0.0001)

print('Start Training')
os.makedirs('./models', exist_ok=True)

for epoch in range(num_epochs):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Using GPU
Start Training
epoch  1


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 1 training loss: 1.990


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 5 training loss: 0.740


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 72 %


100%|██████████| 2500/2500 [00:25<00:00, 98.45it/s]


Accuracy of plane : 83 %
Accuracy of   car : 90 %
Accuracy of  bird : 69 %
Accuracy of   cat : 42 %
Accuracy of  deer : 60 %
Accuracy of   dog : 66 %
Accuracy of  frog : 79 %
Accuracy of horse : 75 %
Accuracy of  ship : 83 %
Accuracy of truck : 76 %
Finished Training


In [10]:
print('Epoch 6 to 15')
for epoch in range(5,15):  # loop over the dataset multiple times
    print('epoch ', epoch + 1)
    train(epoch, trainloader, optimizer, criterion)
    test(testloader, net)
    classwise_test(testloader, net)
    torch.save(net.state_dict(), './models/model-'+str(epoch)+'.pth')

print('Finished Training')

  0%|          | 0/12500 [00:00<?, ?it/s]

Epoch 6 to 15
epoch  6


  0%|          | 0/2500 [00:00<?, ?it/s]

epoch 6 training loss: 0.562


  0%|          | 0/2500 [00:00<?, ?it/s]

...

epoch 15 training loss: 0.052


  0%|          | 0/2500 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 76 %


100%|██████████| 2500/2500 [00:25<00:00, 96.60it/s]


Accuracy of plane : 68 %
Accuracy of   car : 90 %
Accuracy of  bird : 68 %
Accuracy of   cat : 57 %
Accuracy of  deer : 76 %
Accuracy of   dog : 58 %
Accuracy of  frog : 88 %
Accuracy of horse : 84 %
Accuracy of  ship : 92 %
Accuracy of truck : 83 %
Finished Training
